In [19]:
import gym
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

Initialize FrozenLake environment from gym

In [5]:
env = gym.make('FrozenLake-v0')

/Users/b1017579/.local/share/virtualenvs/12._Reinforcement_Learning-DLBnGMow/lib/python3.7/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


Implement a Q learning algorithm

In [10]:
# Initialize table with all zeros.
Q = np.zeros([env.observation_space.n, env.action_space.n])

# Set learning parameters
lr = 0.8
y = 0.9
num_episodes = 2000

# Create list to contain total rewards and steps per episode
rList = []

for i in range(num_episodes):
    #Reset environment and get first new observation
    s = env.reset()
    rAll = 0
    d = False
    j = 0
    #The Q-Table learning algorithm
    while j < 99:
        j+=1
        # Choose an action greedily (with noise) picking from Q table.
        a = np.argmax(Q[s,:] + np.random.randn(1,env.action_space.n)*(1./(i+1)))
        #Get new state and reward from environment
        s1,r,d,_ = env.step(a)
        #Update Q-Table with new knowledge
        Q[s,a] = Q[s,a] + lr*(r + y*np.max(Q[s1,:]) - Q[s,a])
        rAll += r
        s = s1
        if d == True:
            break
    #jList.append(j)
    rList.append(rAll)


In [13]:
print("Score over time: " +  str(sum(rList)/num_episodes))

Score over time: 0.4985


In [16]:
print("Final Q-Table Values")
print(Q)

Final Q-Table Values
[[1.14600814e-01 1.14949334e-03 1.51743473e-03 1.15671259e-03]
 [2.97569392e-04 2.26603815e-05 2.21788056e-04 1.40834221e-01]
 [2.55310622e-02 1.08130063e-03 1.21183265e-03 1.13187512e-03]
 [2.21966514e-04 2.38249115e-04 4.42945624e-05 1.07878204e-03]
 [1.68962401e-01 1.89512158e-04 3.23278069e-04 2.20761674e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [3.02880945e-04 1.66754004e-05 5.34288042e-03 5.12919167e-05]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.49036134e-04 2.06009985e-05 3.67785604e-04 1.93635483e-01]
 [5.03044030e-04 3.80056954e-01 3.33510652e-04 1.04419553e-03]
 [6.82255510e-02 2.16110266e-04 0.00000000e+00 2.20391778e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 7.46441717e-01 2.04714344e-02]
 [0.00000000e+00 9.83397229e-01 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.